In [17]:
# Prolog - Auto Generated #
import os, uuid, matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import pandas

import sys
sys.tracebacklimit = 0

os.chdir(u'C:/Users/friha/PythonEditorWrapper_8de9bb44-c5e2-4b9b-a916-40b4add7c106')
dataset = pandas.read_csv('input_df_889eb8aa-9cdf-4f54-8631-b9d8d6f007b3.csv')

matplotlib.pyplot.figure(figsize=(5.55555555555556,4.16666666666667), dpi=72)
matplotlib.pyplot.show = lambda args=None,kw=None: matplotlib.pyplot.savefig(str(uuid.uuid1()))
# Original Script. Please update your script content here and once completed copy below section back to the original editing window #
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import re

def sanitize_for_patsy(name):
    if not isinstance(name, str): name = str(name)
    name = re.sub(r'[^\w_]', '_', name)
    if name and name[0].isdigit(): name = 's_' + name
    if not name: return "sanitized_empty_val"
    return name

def analyze_feature(df_input, outcome_col, original_feature_col_name,
                    original_reference_category, # This is still needed to guide the model
                    group_name_display,
                    all_categories_ordered=None): # If None, categories are discovered dynamically
    results = []
    default_cols = {'log_OR': np.nan, 'p_value': np.nan,
                    'N_category': 0, 'N_worked_in_category': 0,
                    'N_total_analysis': 0, 'N_worked_total_analysis': 0,
                    'Percentage_worked_in_category': np.nan,
                    'Percentage_worked_total_analysis': np.nan}

    if original_feature_col_name not in df_input.columns:
        print(f"Warning: Feature column '{original_feature_col_name}' not found. Skipping {group_name_display}.")
        results.append({'CharacteristicGroup': group_name_display,
                        'CharacteristicValue': 'Feature column missing', **default_cols})
        return results

    df_feature = df_input[['user_id', outcome_col, original_feature_col_name]].copy()
    df_feature[outcome_col] = pd.to_numeric(df_feature[outcome_col], errors='coerce')
    # Convert feature to string EARLY to ensure consistent type for all operations
    df_feature[original_feature_col_name] = df_feature[original_feature_col_name].astype(str)
    df_feature.dropna(subset=[outcome_col, original_feature_col_name], inplace=True)
    
    if df_feature.empty:
        print(f"Info: No data for {group_name_display} after initial NaN drop for outcome/feature. Skipping model.")
        # If all_categories_ordered is None, we don't know what rows to create.
        # If it's provided, we could create empty rows for them. For now, one error row.
        error_char_value = 'No data after NaN drop'
        if all_categories_ordered: # If an order was provided, list the first one
            error_char_value = f"No data for {all_categories_ordered[0]} (and others)"
        results.append({'CharacteristicGroup': group_name_display,
                        'CharacteristicValue': error_char_value, **default_cols})
        return results

    try:
        df_feature[outcome_col] = df_feature[outcome_col].astype(np.int64)
    except Exception as e_astype:
        print(f"Error converting outcome '{outcome_col}' to np.int64 for {group_name_display}: {e_astype}. Skipping.")
        results.append({'CharacteristicGroup': group_name_display,
                        'CharacteristicValue': f'Outcome conversion error: {e_astype}', **default_cols})
        return results

    df_feature.drop_duplicates(subset=['user_id', original_feature_col_name, outcome_col], inplace=True)

    # Dynamic category discovery if all_categories_ordered is None
    if all_categories_ordered is None:
        # Discover unique categories from the data itself for this feature
        # Ensure they are strings for consistent processing later
        discovered_categories = sorted(df_feature[original_feature_col_name].unique().astype(str))
        categories_to_iterate_for_output = discovered_categories
        if not discovered_categories: # No categories found in data for this feature
             print(f"Info: No unique categories found in data for '{original_feature_col_name}' in group '{group_name_display}'.")
             results.append({'CharacteristicGroup': group_name_display,
                             'CharacteristicValue': 'No categories in data', **default_cols})
             return results
    else: # Use provided order
        categories_to_iterate_for_output = [str(cat) for cat in all_categories_ordered]


    if original_feature_col_name == 'User Experience Group': # Or whatever your exact column name is
        print(f"DEBUG: Unique values for '{original_feature_col_name}' (group: {group_name_display}) being processed: {categories_to_iterate_for_output}")
        print(f"DEBUG: Value counts in filtered df_feature for '{original_feature_col_name}':\n{df_feature[original_feature_col_name].value_counts().head(10)}")


    sanitized_feature_col_name = sanitize_for_patsy(original_feature_col_name)
    df_feature[sanitized_feature_col_name] = df_feature[original_feature_col_name].apply(sanitize_for_patsy)
    sanitized_reference_category = sanitize_for_patsy(original_reference_category)


    if df_feature.empty or df_feature[outcome_col].nunique() < 2 or df_feature[sanitized_feature_col_name].nunique() < 1 :
        error_msg = 'Insufficient data or variance for modeling.'
        # (More detailed error messages)
        print(f"Info: {error_msg} for {group_name_display}. Outputting counts only.")
        for orig_cat_val in categories_to_iterate_for_output: # Use the determined list
            # (Count population logic - same as before)
            N_cat = df_feature[df_feature[original_feature_col_name] == str(orig_cat_val)].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str(orig_cat_val)) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum() if total_n_analysis > 0 else 0
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val),
                'log_OR': np.nan, 'p_value': np.nan,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan
            })
        return results


    actual_sanitized_categories_in_df = df_feature[sanitized_feature_col_name].unique()
    current_sanitized_ref = sanitized_reference_category
    if sanitized_reference_category not in actual_sanitized_categories_in_df:
        # Fallback to mode if specified reference isn't in the (filtered) data for this analysis
        if df_feature[original_feature_col_name].mode().shape[0] > 0:
            mode_original_value = df_feature[original_feature_col_name].mode()[0]
            current_sanitized_ref = sanitize_for_patsy(mode_original_value)
            print(f"Warning: Sanitized reference '{sanitized_reference_category}' (from original '{original_reference_category}') not found for '{group_name_display}'. Using most frequent original value '{mode_original_value}' (sanitized: '{current_sanitized_ref}').")
            if current_sanitized_ref not in actual_sanitized_categories_in_df: # Should not happen if mode is from df_feature
                 print(f"CRITICAL WARNING: Mode '{mode_original_value}' (sanitized: '{current_sanitized_ref}') also not in sanitized categories. Model will likely fail for {group_name_display}")
        else: # No mode means df_feature is effectively empty for this column
            print(f"CRITICAL: No mode available for reference for {group_name_display}. Cannot fit model reliably.")
            # (Fallback to counts for categories_to_iterate_for_output)
            for orig_cat_val in categories_to_iterate_for_output:
                results.append({'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val), **default_cols, 'N_total_analysis': df_feature.shape[0]})
            return results
    sanitized_reference_category = current_sanitized_ref # Use the validated or mode-based reference

    if df_feature[sanitized_feature_col_name].nunique() < 2: # Only one unique predictor category
        print(f"Info: Only one unique predictor category ('{df_feature[original_feature_col_name].unique()[0]}') for {group_name_display}. Outputting counts only.")
        # (Fallback to counts - same as before, but iterate categories_to_iterate_for_output)
        for orig_cat_val in categories_to_iterate_for_output:
            N_cat = df_feature[df_feature[original_feature_col_name] == str(orig_cat_val)].shape[0]
            # ... (rest of count logic)
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str(orig_cat_val)) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum() if total_n_analysis > 0 else 0
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val),
                'log_OR': np.nan, 'p_value': np.nan,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan
            })
        return results

    formula = f"{outcome_col} ~ C({sanitized_feature_col_name}, Treatment(reference='{sanitized_reference_category}'))"
    print(f"Attempting to fit model for {group_name_display} with formula: {formula}")
    try:
        model = smf.logit(formula, data=df_feature).fit(disp=0)
        log_odds = model.params
        p_values = model.pvalues
        # Successful fit
        for orig_cat_val_iter in categories_to_iterate_for_output: # Iterate using determined list
            # (Populate results for successful fit - same as before)
            str_orig_cat_val = str(orig_cat_val_iter) # Ensure it's string for comparison and dict key
            sanitized_cat_val_iter = sanitize_for_patsy(str_orig_cat_val)
            N_cat = df_feature[df_feature[original_feature_col_name] == str_orig_cat_val].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str_orig_cat_val) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum()

            current_log_or_val, current_p_value_val = np.nan, np.nan
            if sanitized_cat_val_iter == sanitized_reference_category:
                pass # Keep as NaN for reference
            else:
                param_name = f"C({sanitized_feature_col_name}, Treatment(reference='{sanitized_reference_category}'))[T.{sanitized_cat_val_iter}]"
                current_log_or_val = log_odds.get(param_name, np.nan)
                current_p_value_val = p_values.get(param_name, np.nan)
            
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str_orig_cat_val,
                'log_OR': current_log_or_val, 'p_value': current_p_value_val,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan
            })

    except Exception as e:
        print(f"Error fitting model or processing results for {group_name_display} (feature: {original_feature_col_name}): {e}. Outputting counts only.")
        # Fallback to counts for ALL categories of this group if model fails
        for orig_cat_val in categories_to_iterate_for_output: # Iterate using determined list
            # (Count population logic - same as before)
            N_cat = df_feature[df_feature[original_feature_col_name] == str(orig_cat_val)].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str(orig_cat_val)) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum() if total_n_analysis > 0 else 0
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val),
                'log_OR': np.nan, 'p_value': np.nan,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan
            })
    return results


# --- Main Script ---
all_results_list = []
outcome_variable_name = 'UserHasWorked'

# Define default columns for error/fallback rows to ensure consistent DataFrame structure
default_error_row_values = {
    'log_OR': np.nan, 'p_value': np.nan,
    'N_category': 0, 'N_worked_in_category': 0,
    'N_total_analysis': 0, 'N_worked_total_analysis': 0,
    'Percentage_worked_in_category': np.nan,
    'Percentage_worked_total_analysis': np.nan
}

if 'user_id' not in dataset.columns or outcome_variable_name not in dataset.columns:
    output_df = pd.DataFrame([{
        'CharacteristicGroup': 'Error',
        'CharacteristicValue': f"'user_id' or '{outcome_variable_name}' missing.",
        **default_error_row_values
    }])
else:
    dataset[outcome_variable_name] = pd.to_numeric(dataset[outcome_variable_name], errors='coerce')
    dataset.dropna(subset=[outcome_variable_name], inplace=True)
    try:
        dataset[outcome_variable_name] = dataset[outcome_variable_name].astype(np.int64)
    except Exception as e_main_astype:
        print(f"CRITICAL ERROR in Main Script: Could not convert outcome variable '{outcome_variable_name}' to np.int64. Error: {e_main_astype}")
        output_df = pd.DataFrame([{
            'CharacteristicGroup': 'Error',
            'CharacteristicValue': f"Failed to process outcome variable '{outcome_variable_name}'.",
            **default_error_row_values
        }])
        # If this critical error happens, output_df is set, and the script will likely end here
        # or the final output_df check will use this one.


    # Only proceed if output_df wasn't set by a critical error above
    if 'output_df' not in locals():
        # 1. Experience Analysis - DYNAMIC LOADING
        experience_reference_category = 'Aucune expérience'
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User Experience Group',
                                           original_reference_category=experience_reference_category,
                                           group_name_display='experience',
                                           all_categories_ordered=None))

        # 2. Rate Analysis
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User Has Min Wage Rate',
                                           original_reference_category=False,
                                           group_name_display='Rate - User Has Min Wage',
                                           all_categories_ordered=None))
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User Has 15 Plus Rate',
                                           original_reference_category=False,
                                           group_name_display='Rate - User Has 15 Plus',
                                           all_categories_ordered=None))

        # 3. Gender Analysis
        if 'User_Gender' in dataset.columns:
            gender_reference_category = 'F'
            all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User_Gender',
                                               original_reference_category=gender_reference_category,
                                               group_name_display='Gender',
                                               all_categories_ordered=None))
        else:
            print("Info: 'User_Gender' column not found. Skipping.")
            all_results_list.append({
                'CharacteristicGroup': 'Gender',
                'CharacteristicValue': 'Column not provided',
                **default_error_row_values
            })

        # 4. Language Analysis
        language_reference = 'NATIVE FRENCH'
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User French Proficiency',
                                           original_reference_category=language_reference,
                                           group_name_display='Codage Langues',
                                           all_categories_ordered=None))

        # 5. City Analysis
        if 'User Is In Paris' in dataset.columns and 'User Is In Bordeaux' in dataset.columns:
            temp_df_city = dataset[['user_id', outcome_variable_name, 'User Is In Paris', 'User Is In Bordeaux']].copy()
            def assign_city_group(row):
                is_paris = False
                if pd.notna(row['User Is In Paris']):
                    if isinstance(row['User Is In Paris'], (bool, np.bool_)): is_paris = row['User Is In Paris']
                    elif isinstance(row['User Is In Paris'], (int, float)): is_paris = bool(row['User Is In Paris'])
                    elif isinstance(row['User Is In Paris'], str): is_paris = row['User Is In Paris'].lower() == 'true'
                is_bordeaux = False
                if pd.notna(row['User Is In Bordeaux']):
                    if isinstance(row['User Is In Bordeaux'], (bool, np.bool_)): is_bordeaux = row['User Is In Bordeaux']
                    elif isinstance(row['User Is In Bordeaux'], (int, float)): is_bordeaux = bool(row['User Is In Bordeaux'])
                    elif isinstance(row['User Is In Bordeaux'], str): is_bordeaux = row['User Is In Bordeaux'].lower() == 'true'
                if is_paris: return 'Paris'
                if is_bordeaux: return 'Bordeaux'
                return 'Other'
            temp_df_city['City_Group_Temp'] = temp_df_city.apply(assign_city_group, axis=1)
            df_paris_bordeaux = temp_df_city[temp_df_city['City_Group_Temp'].isin(['Paris', 'Bordeaux'])].copy()

            if not df_paris_bordeaux.empty and df_paris_bordeaux[outcome_variable_name].notna().all():
                 all_results_list.extend(analyze_feature(df_paris_bordeaux, outcome_variable_name, 'City_Group_Temp',
                                               original_reference_category='Bordeaux',
                                               group_name_display='City',
                                               all_categories_ordered=['Bordeaux', 'Paris']))
            else:
                print("Info: Not enough data for Paris/Bordeaux for City analysis or outcome variable has issues.")
                all_results_list.append({
                    'CharacteristicGroup': 'City',
                    'CharacteristicValue': 'Paris/Bordeaux No Valid Data',
                    **default_error_row_values
                })
        else:
            print("Info: City columns ('User Is In Paris', 'User Is In Bordeaux') not found.")
            all_results_list.append({
                'CharacteristicGroup': 'City',
                'CharacteristicValue': 'Columns not provided',
                **default_error_row_values
            })

        # Final DataFrame creation
        if all_results_list:
            output_df = pd.DataFrame(all_results_list)
            cols_order = ['CharacteristicGroup', 'CharacteristicValue', 'log_OR', 'p_value',
                          'N_category', 'N_worked_in_category', 'Percentage_worked_in_category',
                          'N_total_analysis', 'N_worked_total_analysis', 'Percentage_worked_total_analysis']
            output_df = output_df[[col for col in cols_order if col in output_df.columns]]
        elif 'output_df' not in locals(): # If no critical error set output_df and list is empty
            output_df = pd.DataFrame([{
                'CharacteristicGroup': 'Error',
                'CharacteristicValue': 'No results generated and no critical error caught.',
                **default_error_row_values
            }])

# If output_df was not defined at all (e.g. first check passed but then main_astype failed AND
# the 'output_df' not in locals() check for all_results_list also didn't catch it because an error occurred
# before all_results_list was processed), create a fallback. This is a safeguard.
if 'output_df' not in locals():
    output_df = pd.DataFrame([{
        'CharacteristicGroup': 'Critical Script Failure',
        'CharacteristicValue': 'output_df was not created. Check logs for earlier errors.',
        **default_error_row_values
    }])

output_df 

,CharacteristicGroup,CharacteristicValue,log_OR,p_value,N_category,N_worked_in_category,Percentage_worked_in_category,N_total_analysis,N_worked_total_analysis,Percentage_worked_total_analysis
0,experience,Aucune expérience,NaN,NaN,2234,131,5.863921,37587,2912,7.747359
1,experience,Expérience <1,-0.547586,5.981843e-07,7592,264,3.477345,37587,2912,7.747359
2,experience,Expérience [1-5],0.446012,2.041282e-06,17423,1545,8.867589,37587,2912,7.747359
3,experience,Expérience [5-10],0.649621,9.768420e-11,5330,568,10.656660,37587,2912,7.747359
4,experience,Expérience >10,0.342657,9.772986e-04,5008,404,8.067093,37587,2912,7.747359
5,Rate - User Has Min Wage,False,NaN,NaN,961,5,0.520291,37587,2912,7.747359
6,Rate - User Has Min Wage,True,2.802380,4.260024e-10,36626,2907,7.936985,37587,2912,7.747359
7,Rate - User Has 15 Plus,False,NaN,NaN,30702,2058,6.703146,37587,2912,7.747359
8,Rate - User Has 15 Plus,True,0.678472,7.805901e-56,6885,854,12.403776,37587,2912,7.747359
9,Gender,Column not provided,NaN,NaN,0,0,NaN,0,0,NaN
